In [14]:
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

In [7]:
try: # CUDA-specific install if running on Colab
    %load_ext cudf.pandas
except ModuleNotFoundError:
    print('CuDF not installed, defaulting to regular pandas')
import pandas as pd

CuDF not installed, defaulting to regular pandas


# Preprocessing
As mentioned in our proposal, we intend to:
* Group species into a larger category, instead by genus.
* Discard genus of less than 75 images.
* Apply data augmentation techniques, including rotations, flips, and Gaussian noise.
* Format data for ease of training, validating, and testing.

## Download
This uses the Kaggle API, but you can alternatively direct download. Make sure you have a key at `C:/Users/<username>/.kaggle/kaggle.json` (for Windows machines). We'll be doing our file manipulations in-place since the dataset is fairly large (>3.5 GB).

In [1]:
# download via API
!kaggle datasets download -d anaclaricerezende/mind-funga -p ../data --unzip

Dataset URL: https://www.kaggle.com/datasets/anaclaricerezende/mind-funga
License(s): CC-BY-SA-3.0




  0%|          | 0.00/2.98G [00:00<?, ?B/s]
  0%|          | 1.00M/2.98G [00:00<11:00, 4.84MB/s]
  0%|          | 2.00M/2.98G [00:00<11:16, 4.72MB/s]
  0%|          | 3.00M/2.98G [00:00<10:37, 5.01MB/s]
  0%|          | 4.00M/2.98G [00:00<12:02, 4.42MB/s]
  0%|          | 5.00M/2.98G [00:01<14:15, 3.73MB/s]
  0%|          | 6.00M/2.98G [00:01<12:45, 4.16MB/s]
  0%|          | 7.00M/2.98G [00:01<11:40, 4.55MB/s]
  0%|          | 8.00M/2.98G [00:01<09:43, 5.46MB/s]
  0%|          | 9.00M/2.98G [00:01<08:35, 6.18MB/s]
  0%|          | 10.0M/2.98G [00:02<09:21, 5.67MB/s]
  0%|          | 11.0M/2.98G [00:02<08:53, 5.97MB/s]
  0%|          | 12.0M/2.98G [00:02<08:08, 6.52MB/s]
  0%|          | 13.0M/2.98G [00:02<07:21, 7.21MB/s]
  0%|          | 14.0M/2.98G [00:02<06:45, 7.85MB/s]
  0%|          | 15.0M/2.98G [00:02<06:39, 7.96MB/s]
  1%|          | 17.0M/2.98G [00:02<05:26, 9.73MB/s]
  1%|          | 19.0M/2.98G [00:03<05:26, 9.73MB/s]
  1%|          | 21.0M/2.98G [00:03<05:09, 10.3MB/s]
 

## Genus grouping

In [15]:
def move_files(src: str, dst: str) -> None:
    files = os.scandir(src)
    for file in files:
        shutil.move(file.path, os.path.join(dst, file.name))

In [19]:
# run once, don't touch again
data_path = '../data/MIND.Funga App'
subdir = os.scandir(data_path)

prev_genus = ''
for species in subdir:
    curr_genus = species.name.split(' ', 1)[0]
    new_path = os.path.join(data_path, curr_genus)
    if curr_genus != prev_genus or len(prev_genus) == 0: # if different genus
        prev_genus = curr_genus
        print('Preprocessing ' + curr_genus)
        new_path = os.path.join(data_path, curr_genus)
        os.mkdir(new_path) # create new dir
        move_files(species.path, new_path)
        shutil.rmtree(species.path) # remove old dir
    else: # if same genus
        move_files(species.path, new_path)
        shutil.rmtree(species.path)

Preprocessing Abrachium


OSError: [WinError 145] The directory is not empty: '../data/MIND.Funga App\\Abrachium'